In [53]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [54]:
df_raw = pd.read_csv("Data\\Human activity semi-supervised.csv").drop("subject", axis=1)
test = pd.read_csv("Data\\test.csv").drop("subject", axis=1)
df_labeled = df_raw.copy()

df = df_raw.dropna()
activities = df["Activity"]
test_activites = test["Activity"]
inputs = df.drop("Activity", axis=1)
test_inputs = test.drop("Activity", axis=1)

C:\Users\alexm\AppData\Local\Temp\ipykernel_15288\2170602971.py:1: DtypeWarning: Columns (562) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv("Data\\Human activity semi-supervised.csv").drop("subject", axis=1)


In [55]:
rand_forest = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=42).fit(inputs, activities)
print(classification_report(test_activites, rand_forest.predict(test_inputs)))
print(confusion_matrix(test_activites, rand_forest.predict(test_inputs)))

                    precision    recall  f1-score   support

            LAYING       0.47      0.75      0.58       537
           SITTING       0.16      0.23      0.18       491
          STANDING       0.46      0.27      0.34       532
           WALKING       0.30      0.15      0.20       496
WALKING_DOWNSTAIRS       0.32      0.29      0.30       420
  WALKING_UPSTAIRS       0.54      0.46      0.50       471

          accuracy                           0.37      2947
         macro avg       0.37      0.36      0.35      2947
      weighted avg       0.38      0.37      0.35      2947

[[404  66  66   0   0   1]
 [267 112 107   0   0   5]
 [193 184 146   0   0   9]
 [  0 155   0  76 138 127]
 [  0 105   0 151 123  41]
 [  0 100   0  26 129 216]]


In [56]:
kmeans_model = KMeans(n_clusters=6, random_state=42).fit(inputs)

In [60]:
kmeans_labels = kmeans_model.labels_
centers = kmeans_model.cluster_centers_
Features = inputs.to_numpy()
num_rows, num_cols = Features.shape
dist = [None]*num_rows
for i in range(6):
    for j in range(num_rows):
        dist[j] = np.linalg.norm(centers[i]-Features[j])
    index_min = np.argmin(dist)
    for k in range(len(kmeans_labels)):
        if kmeans_labels[k] == i:
            df_labeled.loc[k, "Activity"] = df_raw.loc[inputs.index[index_min], "Activity"]

inputs_labeled = df_labeled.drop("Activity", axis=1)


In [61]:
conf_matricies = [None]*9
class_rep = [None]*9
for i in range(5,14):
    kmeans_model = KMeans(n_clusters=i).fit(inputs_labeled)
    model = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=42).fit(inputs_labeled, kmeans_model.predict(inputs_labeled))
    conf_matricies[i-5] = confusion_matrix(test_activites, model.predict())